# Contex Manager

Un context manager en python, se define con la sentencia `with` de la siguiente manera

    with_stmt ::=  "with" with_item ("," with_item)* ":" suite
    with_item ::=  expression ["as" target]

Básicamente, un context manages se encarga de inicializar un contexto para ciertas acciones, y asegurarse luego de limpiar, o finalizar el contexto. No es nada esencial, ni que no pueda realizarse mediante contrucciones de programación clásicas, sin embargo, ofrece cierta prácticidad a la hora de realizar ciertos patrones de programación.

Por ejemplo, al abrir un archivo, luego debemos asegurnos de que este ha sido cerrado correctamente. Esto puede realizarse de la siguiente manera:


In [42]:
fd = open('/usr/share/dict/words',mode='r')
try:
    print( fd.readline() )
finally:
    fd.close()


A



En el ejemplo anterior, es importante la contrucción `try ... finally` para asegurarnos de que el archivo sea cerrado, aun en caso de que ocurra algún error.

Esto mismo puede hacerse con la sentencia `with` de forma un poco más consisa.

In [41]:
with open('/usr/share/dict/words',mode='r') as fd:
    print( fd.readline() )


A



Esto es posible debido a que open devuelve un objeto que soporta la interfaz de context manager. Esto sería, una función **\__enter\__** que se ejecuta al entrar al with, y una función **\__exit\__** que se ejecuta al salir, haya excepciones o no. 

Vemos un ejemplo de como implementar un context manager, para realizar transacciones a una base de datos.

In [117]:
class Transaction:
    
    def __init__(self, con):
        ''' 
            Toma una conección a la base de datos como parámetro
        '''
        self.con = con
    
    def __enter__(self):
        ''' 
        La función enter, en caso de que devuelva un valor, lo asociará al targer especificado por 'as' en caso 
        de que haya alguno.
        '''
        self.cur = self.con.cursor()
        return self.cur
    
    def __exit__(self, exc_type, exc_value, traceback):
        '''
        La función exit tomo como parametros el tipo de excepción, el valor, y un traceback. 
        En caso de que no haya error, serán todos nulos.
        '''
        if exc_type:
            print('Hubo un error: rollback!')
            self.con.rollback()
        else:
            print('Todo bien: commit!')
            self.con.commit()
        
        self.cur.close()
        return True
        



Para probarlo, utilizaremos el módulo `sqlite3` de la standard library y crearemos un base de datos en memoría, con una tabla _people_

In [114]:
import sqlite3

con = sqlite3.connect(":memory:")
cur = con.cursor()
cur.execute("drop table if exists  people")
cur.execute("create table people (name, age)")
cur.close()
con.commit()


Todo lo que se realice dentro de la transacción que definimo, será commiteado a la base, salvo que ocurra algún error, en cuyo caso, se hará un rollback.

In [115]:
with Transaction(con) as cur:
    cur.execute("insert into people values (?, ?)", ('pedro', 20))
    raise Exception('ocurrio un error')
    cur.execute("insert into people values (?, ?)", ('juan', 18))


Hubo un error: rollback!


Consultamos la base para ver si los resultados llegaron a commitearse. 

In [112]:
cur = con.cursor()
cur.execute("select name, age from people")
for row in cur:
    print(row)


## Referencias

* [PEP](https://www.python.org/dev/peps/pep-0343/)
* [contextlib](https://docs.python.org/3/library/contextlib.html)